In [1]:
import os
os.chdir(r'C:\Users\imran\anaconda3\envs\ZohaibLabs')

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input, SimpleRNN
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [5]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [7]:
def create_rnn():
    input_data = Input(shape=(time_steps, num_features))
    rnn_layer1 = SimpleRNN(8, return_sequences=True)(input_data)
    rnn_layer2 = SimpleRNN(20)(rnn_layer1)
    x = Flatten()(rnn_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [9]:
model1 = create_rnn()
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 24, 8)               │             240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 20)                  │             580 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 841 (3.29 KB)

 Trainable params: 841 (3.29 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [24]:
tensorflow.keras.utils.plot_model(model1 )

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [28]:
checkpoints = r'C:\Users\imran\anaconda3\envs\ZohaibLabs\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\imran\anaconda3\envs\lab9'
#FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
#JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [30]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)

# construct the set of callbacks
callbacks = [EpochCheckpoint]

In [32]:
print("[INFO] compiling model...")
model =create_rnn()
opt = Adam(1e-3)
model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])



[INFO] compiling model...


In [36]:
import os
path_dataset =r'C:\Users\imran\anaconda3\envs\lab789'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\imran\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [38]:
time_steps=24
num_features=21

In [44]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.40350866317749023 sec


In [46]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/10
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0107 - mae: 0.0107 - mape: 187.1777
Epoch 1: val_loss did not improve from 0.01127
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 0.0107 - mae: 0.0107 - mape: 187.4510 - val_loss: 0.0133 - val_mae: 0.0133 - val_mape: 7.1201
Epoch 2/10
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0105 - mae: 0.0105 - mape: 21.7226
Epoch 2: val_loss did not improve from 0.01127
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0105 - mae: 0.0105 - mape: 22.3803 - val_loss: 0.0141 - val_mae: 0.0141 - val_mape: 6.9914
Epoch 3/10
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0102 - mae: 0.0102 - mape: 19.5192
Epoch 3: val_loss improved from 0.01127 to 0.00915, saving model to C:\Users\imran\anaconda3\envs\ZohaibLabs\E1-cp-0003-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0102 - mae: 0.0102 - mape: 19.8665 - val_loss: 0.0092 - val_mae: 0.0092 - val_mape: 3.9347
Epoch 4/10
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0100 - mae: 0.0100 - mape: 176.2750
Epoch 4: val_loss did not improve from 0.00915
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0100 - mae: 0.0100 - mape: 176.3884 - val_loss: 0.0103 - val_mae: 0.0103 - val_mape: 4.4848
Epoch 5/10
2641/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0097 - mae: 0.0097 - mape: 23.9700
Epoch 5: val_loss did not improve from 0.00915
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0097 - mae: 0.0097 - mape: 25.0923 - val_loss: 0.0092 - val_mae: 0.0092 - val_mape: 3.8612
Epoch 6/10
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0095 - mae: 0.0095 - mape: 148.7399
Epoch 6: val_loss improved from 0.00915 to 0.00904, saving model to C:\Users\imran\anaconda3\envs\ZohaibLabs\E1-cp-0006-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0095 - mae: 0.0095 - mape: 149.0751 - val_loss: 0.0090 - val_mae: 0.0090 - val_mape: 4.3845
Epoch 7/10
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0093 - mae: 0.0093 - mape: 84.8578
Epoch 7: val_loss improved from 0.00904 to 0.00825, saving model to C:\Users\imran\anaconda3\envs\ZohaibLabs\E1-cp-0007-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 0.0093 - mae: 0.0093 - mape: 85.1630 - val_loss: 0.0082 - val_mae: 0.0082 - val_mape: 3.4242
Epoch 8/10
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0092 - mae: 0.0092 - mape: 180.4580
Epoch 8: val_loss did not improve from 0.00825
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0092 - mae: 0.0092 - mape: 180.4716 - val_loss: 0.0085 - val_mae: 0.0085 - val_mape: 3.6237
Epoch 9/10
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0091 - mae: 0.0091 - mape: 1065.5612
Epoch 9: val_loss improved from 0.00825 to 0.00794, saving model to C:\Users\imran\anaconda3\envs\ZohaibLabs\E1-cp-0009-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0091 - mae: 0.0091 - mape: 1063.6757 - val_loss: 0.0079 - val_mae: 0.0079 - val_mape: 3.3519
Epoch 10/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0090 - mae: 0.0090 - mape: 352.4886
Epoch 10: val_loss did not improve from 0.00794
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0090 - mae: 0.0090 - mape: 352.4519 - val_loss: 0.0096 - val_mae: 0.0096 - val_mape: 4.5061


In [56]:
import numpy as np
from keras.models import load_model
from keras.metrics import MeanAbsoluteError  # ✅ This line fixes the NameError

# Load the model with the proper custom_objects argument
model = load_model(
    r'C:\Users\imran\anaconda3\envs\ZohaibLabs\E1-cp-0009-loss0.01.h5',
    custom_objects={'mae': MeanAbsoluteError()}
)

# Predict and inverse transform
y_pred_scaled = model.predict(test_X)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)

# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squared Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred) / y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred) / y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

# Shapes
print('\n\ny_test_unscaled.shape= ', y_test_unscaled.shape)
print('y_pred.shape= ', y_pred.shape)


379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Mean Absolute Error (MAE): 123.03
Median Absolute Error (MedAE): 97.86
Mean Squared Error (MSE): 26038.89
Root Mean Squared Error (RMSE): 161.37
Mean Absolute Percentage Error (MAPE): 0.84 %
Median Absolute Percentage Error (MDAPE): 0.67 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [58]:
checkpoints = r'C:\Users\imran\anaconda3\envs\lab9\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\imran\anaconda3\envs\lab9\E1-cp-0053-loss0.01.h5'
start_epoch= 54

In [66]:
import os
import numpy as np
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K
from keras.losses import MeanAbsoluteError  # if using in custom_objects

# Assuming PC is your model builder class, and checkpoints is the filepath to save
from your_model_file import PC  # replace with the actual file where PC is defined

# Define your checkpoint path and model file
checkpoints = 'E1-cp-best-loss0.01.h5'
model_path = r'C:\Users\imran\anaconda3\envs\ZohaibLabs\E1-cp-0009-loss0.01.h5'

# Construct the callback to save only the *best* model to disk
EpochCheckpoint1 = ModelCheckpoint(
    checkpoints,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

# Construct the set of callbacks
callbacks = [EpochCheckpoint1]

# Check if the model file exists
if not os.path.exists(model_path):
    print("[INFO] model file not found. Compiling a new model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])
else:
    print(f"[INFO] loading model from: {model_path}")
    model = load_model(model_path, custom_objects={'mae': MeanAbsoluteError()})
    
    # Update learning rate
    print("[INFO] old learning rate:", K.get_value(model.optimizer.lr))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate:", K.get_value(model.optimizer.lr))


ModuleNotFoundError: No module named 'your_model_file'

In [68]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

AttributeError: 'str' object has no attribute 'fit'

In [19]:

model = load_model(r'C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab9\E2-cp-0006-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 1s 1ms/step
Mean Absolute Error (MAE): 144.75
Median Absolute Error (MedAE): 113.65
Mean Squared Error (MSE): 36189.68
Root Mean Squared Error (RMSE): 190.24
Mean Absolute Percentage Error (MAPE): 0.98 %
Median Absolute Percentage Error (MDAPE): 0.79 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
